In [2]:
import faiss
import numpy as np
import json
import torch
from transformers import AutoTokenizer, AutoModel

In [3]:
# Load the embeddings and metadata
embeddings = np.load('embeddings.npy')
with open('metadata.json', 'r', encoding='utf-8') as meta_file:
    metadata = json.load(meta_file)

# Create the FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Determine the appropriate device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2').to(device)
print("Model and tokenizer loaded successfully on", device)

Model and tokenizer loaded successfully on cpu


: 

In [3]:
def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    return embeddings

def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        model_output = model(**inputs)
        embeddings = model_output.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    # Clear GPU memory
    torch.cuda.empty_cache()
    return embeddings

@app.route('/')
def memory_test():
    query = "attent"
    query_embedding = embed_text(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, k=5)
    results = [metadata[idx] for idx in indices[0]]
    return {'results': results}


: 

In [ ]:
def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)
    print(f"Input shape: {inputs['input_ids'].shape}")
    with torch.no_grad():
        model_output = model(**inputs)
        embeddings = model_output.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
        print(f"Embedding shape: {embeddings.shape}")
    return embeddings

In [8]:
query = "attent"
query_embedding = embed_text(query).reshape(1, -1)
distances, indices = index.search(query_embedding, k=5)
results = [metadata[idx] for idx in indices[0]]
results

: 

In [ ]:
def search():
    query = request.json.get('query')
    query_embedding = embed_text(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, k=5)
    
    results = [metadata[idx] for idx in indices[0]]
    return jsonify(results)

In [ ]:
def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    return embeddings

@bp.route('/search', methods=['POST'])
def search():
    query = request.json.get('query')
    query_embedding = embed_text(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, k=5)
    
    results = [metadata[idx] for idx in indices[0]]
    return jsonify(results)

In [ ]:
from flask import Blueprint, request, jsonify
import faiss
import numpy as np
import json
import torch
from transformers import AutoTokenizer, AutoModel

bp = Blueprint('search', __name__)

# Load the embeddings and metadata
embeddings = np.load('embeddings.npy')
with open('metadata.json', 'r', encoding='utf-8') as meta_file:
    metadata = json.load(meta_file)

# Create the FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Load the tokenizer and model for generating query embeddings
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    return embeddings

@bp.route('/search', methods=['POST'])
def search():
    query = request.json.get('query')
    query_embedding = embed_text(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, k=5)
    
    results = [metadata[idx] for idx in indices[0]]
    return jsonify(results)
